# Foundations of AI/ML by IIIT-Hyderabad & Talent Sprint
# Lab08 Experiment 01

## MLP and Backpropagation ##

## Training a MLP

Here, we will build a 3-layer neural network with one input layer, one hidden layer, and one output layer. The number of nodes in the input layer is determined by the dimensionality of our data. Similarly, the number of nodes in the output layer is determined by the number of classes we have.

### How our network makes predictions

Our network makes predictions using *forward propagation*, which is just a bunch of matrix multiplications and the application of the activation function(s) we defined above. If $x$ is the $N$-dimensional input to our network then we calculate our prediction $\hat{y}$ (of lets say dimension $C$) as follows:

$$
\begin{aligned}
z_1 & = xW_1 + b_1 \\
a_1 & = \tanh(z_1) \\
z_2 & = a_1W_2 + b_2 \\
a_2 & = \hat{y} = \mathrm{softmax}(z_2)
\end{aligned}
$$

$z_i$ is the weighted sum of inputs of layer $i$ (bias included) and $a_i$ is the output of layer $i$ after applying the activation function. $W_1, b_1, W_2, b_2$ are  parameters of our network, which we need to learn from our training data. You can think of them as matrices transforming data between layers of the network. Looking at the matrix multiplications above we can figure out the dimensionality of these matrices. If we use 100 nodes for our hidden layer then $W_1 \in \mathbb{R}^{N\times100}$, $b_1 \in \mathbb{R}^{100}$, $W_2 \in \mathbb{R}^{100\times C}$, $b_2 \in \mathbb{R}^{C}$. Now you see why we have more parameters if we increase the size of the hidden layer.

### Learning the Parameters/Backpropagation

Learning the parameters for our network means finding parameters ($W_1, b_1, W_2, b_2$) that minimize the error on our training data. But how do we define the error? We call the function that measures our error the *loss function*. A common choice with the softmax output is the cross-entropy loss. If we have $N$ training examples and $C$ classes then the loss for our prediction $\hat{y}$ with respect to the true labels $y$ is given by:

$$
\begin{aligned}
L(y,\hat{y}) = - \frac{1}{N} \sum_{n \in N} \sum_{i \in C} y_{n,i} \log\hat{y}_{n,i}
\end{aligned}
$$

The formula looks complicated, but all it really does is sum over our training examples and add to the loss if we predicted the incorrect class. So, the further away $y$ (the correct labels) and $\hat{y}$ (our predictions) are, the greater our loss will be. 

Remember that our goal is to find the parameters that minimize our loss function. We can use gradient descent to find its minimum. Here, we implement the most vanilla version of gradient descent, also called batch gradient descent with a fixed learning rate. Variations such as SGD (stochastic gradient descent) or minibatch gradient descent typically perform better in practice but we are not going into that in this experiment.

As an input, gradient descent needs the gradients (vector of derivatives) of the loss function with respect to our parameters: $\frac{\partial{L}}{\partial{W_1}}$, $\frac{\partial{L}}{\partial{b_1}}$, $\frac{\partial{L}}{\partial{W_2}}$, $\frac{\partial{L}}{\partial{b_2}}$. To calculate these gradients we use the famous *backpropagation algorithm*, which is a way to efficiently calculate the gradients starting from the output.

Applying the backpropagation formula using chain rule we find the following:

$$
\begin{aligned}
& \delta_3 = \frac{\partial{L}}{\partial{z_2}} = \frac{\partial{L}}{\partial{a_2}}\times\frac{\partial{a_2}}{\partial{z_2}} = -(y - \hat{y})\\
\end{aligned}
$$
where $a_2$ is $\hat{y}$
$$
\begin{aligned}
& \frac{\partial{L}}{\partial{W_2}} = \frac{\partial{L}}{\partial{z_2}}\times\frac{\partial{z_2}}{\partial{W_2}} = a_1^T \delta_3  \\
& \frac{\partial{L}}{\partial{b_2}} = \frac{\partial{L}}{\partial{z_2}}\times\frac{\partial{z_2}}{\partial{b_2}} = \delta_3\\
& \delta_2 = \frac{\partial{L}}{\partial{z_1}} = \frac{\partial{L}}{\partial{z_2}}\times\frac{\partial{z_2}}{\partial{a_1}}\times\frac{\partial{a_1}}{\partial{z_1}} = (1 - \tanh^2z_1) \circ \delta_3W_2^T \\
& \frac{\partial{L}}{\partial{W_1}} = \frac{\partial{L}}{\partial{z_1}}\times\frac{\partial{z_1}}{\partial{W_1}} = x^T \delta_2\\
& \frac{\partial{L}}{\partial{b_1}} = \frac{\partial{L}}{\partial{z_1}}\times\frac{\partial{z_1}}{\partial{b_1}} = \delta_2 \\
\end{aligned}
$$

$\delta_3 = $ derivative of cross-entropy loss with Softmax as Activation [We will not go into its derivation]


### 1. Loading the dataset

In [25]:
import numpy as np
from scipy import ndimage
from matplotlib import pyplot as plt
from sklearn import manifold, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

#Load MNIST dataset 
digits = datasets.load_digits(n_class=10)
# Create our X and y data
X = digits.data
Y = digits.target
print(X.shape, Y.shape)
num_examples = X.shape[0]      ## training set size
nn_input_dim = X.shape[1]      ## input layer dimensionality
nn_output_dim = len(np.unique(Y))       ## output layer dimensionality

params = {
    "lr":1e-5,        ## learning_rate
    "max_iter":1000,
    "h_dimn":40,     ## hidden_layer_size
}

(1797, 64) (1797,)


### 2. Writing helper functions for the MLP

In [41]:
def build_model():
    hdim = params["h_dimn"]
    # Initialize the parameters to random values.
    np.random.seed(0)
    W1 = np.random.randn(nn_input_dim, hdim) / np.sqrt(nn_input_dim)
    b1 = np.zeros((1, hdim))
    W2 = np.random.randn(hdim, nn_output_dim) / np.sqrt(hdim)
    b2 = np.zeros((1, nn_output_dim))

    # This is what we return at the end
    model = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
    return model

def softmax(x):
    exp_scores = np.exp(x)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    return probs

def feedforward(model, x):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    z1 = x.dot(W1) + b1
    a1 = np.tanh(z1)
    z2 = a1.dot(W2) + b2
    probs = softmax(z2)
    return a1, probs

def backpropagation(model, x, y, a1, probs):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    
    delta3 = probs
    delta3[range(y.shape[0]), y] -= 1
    dW2 = (a1.T).dot(delta3)
    db2 = np.sum(delta3, axis=0, keepdims=True)
    delta2 = delta3.dot(W2.T) * (1 - np.power(a1, 2))
    dW1 = np.dot(x.T, delta2)
    db1 = np.sum(delta2, axis=0)
    return dW2, db2, dW1, db1

def calculate_loss(model, x, y):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    
    # Forward propagation to calculate predictions
    _, probs = feedforward(model, x)
    
    # Calculating the cross entropy loss
    corect_logprobs = -np.log(probs[range(y.shape[0]), y])
    data_loss = np.sum(corect_logprobs)
    
    return 1./y.shape[0] * data_loss

def test(model, x, y):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    # Forward propagation to calculate predictions
    _, probs = feedforward(model, x)
    preds = np.argmax(probs, axis=1)
    return np.count_nonzero(y==preds)/y.shape[0]

def train(model, X_train, X_test, Y_train, Y_test, print_loss=True):
    # Gradient descent. For each batch...
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    for i in range(0, params["max_iter"]):

        # Forward propagation
        a1, probs = feedforward(model, X_train)

        # Backpropagation
        dW2, db2, dW1, db1 = backpropagation(model, X_train, Y_train, a1, probs)

        # Gradient descent parameter update
        W1 += -params["lr"] * dW1
        b1 += -params["lr"] * db1
        W2 += -params["lr"] * dW2
        b2 += -params["lr"] * db2
        
        # Assign new parameters to the model
        model = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
        if print_loss and i % 50 == 0:
            print("Loss after iteration %i: %f" %(i, calculate_loss(model, X_train, Y_train)),
                  ", Test accuracy:", test(model, X_test, Y_test), "\n")
    return model

### 3. Training the model

In [42]:
model = build_model()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5)

model = train(model, X_train, X_test, Y_train, Y_test)

Loss after iteration 0: 2.492749 , Test accuracy: 0.13236929922135707 

Loss after iteration 50: 1.712895 , Test accuracy: 0.45050055617352613 

Loss after iteration 100: 1.311046 , Test accuracy: 0.6596218020022246 

Loss after iteration 150: 1.043388 , Test accuracy: 0.7686318131256952 

Loss after iteration 200: 0.858650 , Test accuracy: 0.8131256952169077 

Loss after iteration 250: 0.725068 , Test accuracy: 0.8498331479421579 

Loss after iteration 300: 0.625338 , Test accuracy: 0.8654060066740823 

Loss after iteration 350: 0.548757 , Test accuracy: 0.882091212458287 

Loss after iteration 400: 0.486890 , Test accuracy: 0.8921023359288098 

Loss after iteration 450: 0.436110 , Test accuracy: 0.8987764182424917 

Loss after iteration 500: 0.392825 , Test accuracy: 0.9087875417130145 

Loss after iteration 550: 0.355599 , Test accuracy: 0.917686318131257 

Loss after iteration 600: 0.324155 , Test accuracy: 0.9210233592880979 

Loss after iteration 650: 0.297017 , Test accuracy: 0.